This notebook pulls 2010 census data and attempts to generate a large set of points that approximates a smooth surface

In [1]:
# Declare static variables

n=5 # The number of points to assign to each census block
BINOMIAL_TRIALS = 40 # The number of trials in the binomial distribution used for weighting points in blocks. The higher the value, the more evenly distributed the population points will be through the census block
BINOMIAL_SUCCESS = 0.5 # The probability of success for each trial in he weight assignment. Must be <=1. Use 1 for a uniform distribution

# Apply a transformation to make the outcome more normal
# Apply a normal distribution and use min/max normalization
# Weighted Poisson binomial distribution


# Binomial distribution is nearly normal if np(1-p) >= 10
print(BINOMIAL_TRIALS * BINOMIAL_SUCCESS * (1 - BINOMIAL_SUCCESS) >= 10)

True


In [2]:
# Import libraries

import pandas as pd
import geopandas
import numpy as np
import requests
from io import BytesIO
import folium
from IPython.display import clear_output
from itertools import chain
from shapely.ops import nearest_points

In [3]:
# Pull school location data from the open data portal

data_url = 'https://data.delaware.gov/resource/p3ez-si4g.geojson'

data = requests.get(data_url, stream=True)
schools = geopandas.GeoDataFrame.from_file(BytesIO(data.content))

In [4]:
# Visualize schools on a map

# initialize the map and store it in a folium map object
us_map = folium.Map(location=[38.9108, -75.5277], zoom_start=8, tiles=None)

# Add background tiles
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(us_map)

# Style and highlight functions map population values to color values
style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':'red', 
                            'fillOpacity':0.75}

# Add a map over the tiles with the given colors and a tooltip
NIL=folium.features.GeoJson(
        schools, # Full geopandas data
        style_function=style_function, # function for base colors
        control=False
    )

# Add elements to map
us_map.add_child(NIL)

In [5]:
# TODO: Use geolocator on missed addresses

# Filter schools with no coordinates
schools = schools.loc[~schools["geometry"].isnull()]

In [6]:
# Request shapefile data for 2010 census tracts and convert to geopandas dataframe

# Shapefile url
data_url = 'https://www2.census.gov/geo/tiger/GENZ2010/gz_2010_10_140_00_500k.zip'


# Request data
data = requests.get(data_url)
# convert to pandas dataframe
tract_data = geopandas.read_file(BytesIO(data.content))

In [7]:
# Request shapefile data for 2010 census tracts and convert to geopandas dataframe

# Shapefile url
data_url = 'https://www2.census.gov/geo/tiger/TIGER2010/TABBLOCK/2010/tl_2010_10_tabblock10.zip'


# Request data
data = requests.get(data_url)
# convert to pandas dataframe
block_data = geopandas.read_file(BytesIO(data.content))

In [8]:
# For each census block, create a bounding box
block_bounds = block_data["geometry"].bounds

# Attch GEOID to boundaries
block_bounds = block_data[["GEOID10","geometry"]].merge(block_bounds, left_index=True, right_index=True)

In [13]:
schools

,districtcode,org_county,districttype,districtname,organization_name,org_street2,org_state,org_street1,org_zip,schoolcode,lowestgrade,org_city,schooltype,schoolyear,highestgrade,geometry
0,0,Kent,Not Applicable,State of Delaware,State of Delaware,Suite 2,DE,401 Federal Street,199013639,0,PK,Dover,Not Applicable,2015,12,POINT (-75.52246 39.15785)
1,10,Kent,Regular,Caesar Rodney School District,Caesar Rodney School District,None,DE,7 Front Street,19934,0,PK,Wyoming,Not Applicable,2015,12,POINT (-75.55275 39.11954)
2,10,Kent,Regular,Caesar Rodney School District,Allen Frear Elementary School,None,DE,238 Sorghum Mill Road,19934,610,01,Camden,PUBLIC,2015,05,POINT (-75.51269 39.10247)
5,10,Kent,Regular,Caesar Rodney School District,Kent Elementary Intensive Learning Center,None,DE,5 Old North Road,199341247,615,KG,Camden,PUBLIC,2015,05,POINT (-75.55001 39.11888)
6,10,Kent,Regular,Caesar Rodney School District,Nellie Hughes Stokes Elementary School,None,DE,3874 Upper King Road,19904,616,01,Dover,PUBLIC,2015,05,POINT (-75.55400 39.10176)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,36,Sussex,Regular,Indian River School District,Georgetown Middle School,None,DE,301 West Market Street,19947,743,06,Georgetown,PUBLIC,2018,08,POINT (-75.39140 38.68560)
996,36,Sussex,Regular,Indian River School District,Indian River High School,None,DE,29772 Armory Road,19939,746,08,Dagsboro,PUBLIC,2018,12,POINT (-75.22965 38.54485)
997,36,Sussex,Regular,Indian River School District,Sussex Central High School,None,DE,26026 Patriots Way,19947,748,09,Georgetown,PUBLIC,2018,12,POINT (-75.32423 38.63076)
998,36,Sussex,Regular,Indian River School District,Southern Delaware School of the Arts,None,DE,27 Hosier Street,19975,749,KG,Selbyville,PUBLIC,2018,08,POINT (-75.22520 38.46050)


In [14]:
# Fit a 2D Gaussian distribution over the bounding boxes

# Takes in a row of 'block_bounds' and outputs a 2D Gaussian distribution of 'n' points over the bounding box, as well as the GEOID
def get_points(row,n):
    print(f"Processing Block {row['GEOID10']}...")
    # 'i' is the total number of points left to assign
    i=n
    # 'points_return' is the list of all points for the block
    # TODO: CRS is hardcoded
    points_return = geopandas.GeoSeries(crs="EPSG:4269")
    # Allocate points until n have been assigned
    while i > 0:
        # Generates a uniform distribution for the y-axis located at the center of the box
        pointsy = np.random.uniform(low=row["miny"], high=row["maxy"], size=i)
        # Generates a uniform distribution for the x-axis located at the center of the box
        pointsx = np.random.uniform(low=row["minx"], high=row["maxx"], size=i)
        # Convert the points to Shapely points
        points = geopandas.GeoSeries(geopandas.points_from_xy(pointsx, pointsy, crs="EPSG:4269"))
        # Check if the points are inside the block
        point_checks = points.within(row["geometry"])
        # Add found points to our list
        points_return = geopandas.GeoSeries(pd.concat([points_return, points[point_checks]], ignore_index=True), crs=points_return.crs)
        # Set 'i' equal to the number of missed points
        i = n - points_return.size
        
    # Generate a series containing the distance from each point to the nearest school
    distances = points_return.apply(lambda x: schools.distance(x).min())
    # Normalize the set of distances so they sum to 1
    distances = distances / distances.sum()
    # Clear warnings from notebook output to prevent crash
    clear_output()
    # Return an array with every point in the cloud, the weights for each point and the GEOID
    return list(chain(points_return.values, distances.values, [row["GEOID10"]]))
    

In [15]:
# Fit a Gaussian distribution to each block
point_cloud = block_bounds.apply(get_points, axis=1, args=(n,), result_type='expand')

# Rename columns of the pointcloud
point_cloud.columns = ['point_' + str(x) if x<n else 'weight_' + str(x-n) if x<2*n else 'GEOID' for x in point_cloud.columns]

point_cloud

,point_0,point_1,point_2,point_3,point_4,weight_0,weight_1,weight_2,weight_3,weight_4,GEOID
0,POINT (-75.48697559282309 39.12118494125572),POINT (-75.48585908543907 39.12186381760254),POINT (-75.4867271809566 39.12176064653224),POINT (-75.48470081465994 39.121961572685336),POINT (-75.48680039852336 39.12139019902948),0.194377,0.202680,0.197831,0.209062,0.196051,100010411001014
1,POINT (-75.48955466592558 39.12218941587317),POINT (-75.49170475101178 39.121890137467425),POINT (-75.48955101540818 39.12229288561236),POINT (-75.49031755680731 39.12136412852302),POINT (-75.49077894329326 39.122235979014484),0.205265,0.192567,0.205762,0.197388,0.199019,100010411001007
2,POINT (-75.48182772458908 39.12023438394083),POINT (-75.48195777953202 39.12013186078736),POINT (-75.48255903497656 39.120272777381636),POINT (-75.48284695037104 39.119857274748355),POINT (-75.48257891388675 39.1190637418341),0.203653,0.202680,0.200010,0.197290,0.196367,100010411001018
3,POINT (-75.4389620798654 39.10577307408306),POINT (-75.440561273781 39.10401696633933),POINT (-75.43651159183108 39.10658755733187),POINT (-75.43978224085643 39.10263127542011),POINT (-75.44070136164662 39.103645111296544),0.204512,0.194676,0.214334,0.193275,0.193203,100010432021103
4,POINT (-75.52861244578528 39.16636695856447),POINT (-75.52872565587192 39.16663870920683),POINT (-75.52808303881791 39.16588035603424),POINT (-75.52852870910671 39.16654939811131),POINT (-75.52863986208713 39.16676364854422),0.201662,0.207290,0.183366,0.201285,0.206396,100010409001039
...,...,...,...,...,...,...,...,...,...,...,...
24110,POINT (-75.39259092542254 38.54371092268125),POINT (-75.41050884601898 38.54817499252541),POINT (-75.40289793452686 38.54935913409326),POINT (-75.40874519209999 38.53694100779404),POINT (-75.40479862519827 38.54014639522326),0.187330,0.202141,0.191594,0.213716,0.205219,100050517011022
24111,POINT (-75.46227671747472 38.53827697480899),POINT (-75.45721759031149 38.53564003818175),POINT (-75.45438887135651 38.54017459465918),POINT (-75.45618383152978 38.54352270381485),POINT (-75.45302282600272 38.5358501894),0.189958,0.199140,0.203817,0.200553,0.206532,100050517011047
24112,POINT (-75.45505957910697 38.5339219328395),POINT (-75.45525422556719 38.53477589123094),POINT (-75.45771341763368 38.535028456411055),POINT (-75.45218727937439 38.53272639970359),POINT (-75.45285838310338 38.53381282293307),0.199243,0.198800,0.194510,0.204379,0.203068,100050517011049
24113,POINT (-75.46027793505081 38.529837481364446),POINT (-75.45941804606336 38.527930892949946),POINT (-75.45773089240562 38.529986938333316),POINT (-75.45901987599234 38.53146028597175),POINT (-75.45866810185743 38.52908309263055),0.197710,0.199668,0.202231,0.199646,0.200745,100050517021120


In [16]:
# Pull population data for 2010 Census blocks
# Define request parameters

year = '2010' # Year of interest
datasource = 'dec' # Survey name
subsource = 'pl' # Subsurvey name
GET = 'P001001,H001001,P001003' # Variables to query
FOR = 'block:*' # for predicate
IN = 'state:10&in=county:*&in=tract:*'

# Filepath to your Census API key
keyfile = 'CensusAPIKey.txt'

# Formatted API call
data_url = f'https://api.census.gov/data/{year}/{datasource}/{subsource}?get={GET}&for={FOR}&in={IN}'

# Read Census key into 'api_key'
with open(keyfile) as key:
    api_key = key.read().strip()

# Add key to url
data_url = f'{data_url}&key={api_key}'

# Request data and convert from json
data = requests.get(data_url).json()
# First entry in list is a list of variable names
data = pd.DataFrame(data[1:], columns = data[0])

# Rename columns to match shapefile pull
data.rename(columns = {"state":"STATEFP10", "county":"COUNTYFP10", "tract":"TRACTCE10", "block":"BLOCKCE10"}, inplace=True)

# Attach to block shapes
block_data = block_data.merge(data, on=["STATEFP10","COUNTYFP10","TRACTCE10","BLOCKCE10"])

In [17]:
# Pull population data for 2010 Census tracts
# Define request parameters

year = '2010' # Year of interest
datasource = 'dec' # Survey name
subsource = 'pl' # Subsurvey name
GET = 'P001001,H001001,P001003' # Variables to query
FOR = 'tract:*' # for predicate
IN = 'state:10' # in predicate


# Filepath to your Census API key
keyfile = 'CensusAPIKey.txt'

# Formatted API call
data_url = f'https://api.census.gov/data/{year}/{datasource}/{subsource}?get={GET}&for={FOR}&in={IN}'

# Read Census key into 'api_key'
with open(keyfile) as key:
    api_key = key.read().strip()

# Add key to url
data_url = f'{data_url}&key={api_key}'

# Request data and convert from json
data = requests.get(data_url).json()
# First entry in list is a list of variable names
data = pd.DataFrame(data[1:], columns = data[0])

# Rename columns to match shapefile pull
data.rename(columns = {"state":"STATE", "county":"COUNTY", "tract":"TRACT"}, inplace=True)

# Attach to tract shapes
tract_data = tract_data.merge(data, on=["STATE","COUNTY","TRACT"])

In [18]:
block_data

,STATEFP10,COUNTYFP10,TRACTCE10,BLOCKCE10,GEOID10,NAME10,MTFCC10,UR10,UACE10,UATYP10,FUNCSTAT10,ALAND10,AWATER10,INTPTLAT10,INTPTLON10,geometry,P001001,H001001,P001003
0,10,001,041100,1014,100010411001014,Block 1014,G5040,U,24580,U,S,50816,0,+39.1216358,-075.4858233,"POLYGON ((-75.48486 39.12239, -75.48481 39.122...",244,77,187
1,10,001,041100,1007,100010411001007,Block 1007,G5040,U,24580,U,S,48931,0,+39.1219647,-075.4904073,"POLYGON ((-75.49019 39.12340, -75.49005 39.123...",167,50,135
2,10,001,041100,1018,100010411001018,Block 1018,G5040,U,24580,U,S,28485,0,+39.1196396,-075.4826429,"POLYGON ((-75.48313 39.11849, -75.48333 39.118...",33,10,21
3,10,001,043202,1103,100010432021103,Block 1103,G5040,R,None,None,S,160376,0,+39.1054024,-075.4393957,"POLYGON ((-75.44219 39.10734, -75.43890 39.107...",14,6,12
4,10,001,040900,1039,100010409001039,Block 1039,G5040,U,24580,U,S,12254,0,+39.1662742,-075.5285219,"POLYGON ((-75.52849 39.16525, -75.52862 39.165...",53,27,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24110,10,005,051701,1022,100050517011022,Block 1022,G5040,R,None,None,S,2614704,0,+38.5418205,-075.4026864,"POLYGON ((-75.41186 38.54192, -75.41169 38.542...",1,1,1
24111,10,005,051701,1047,100050517011047,Block 1047,G5040,R,None,None,S,1331210,0,+38.5406585,-075.4582756,"POLYGON ((-75.45237 38.54473, -75.45222 38.544...",72,29,72
24112,10,005,051701,1049,100050517011049,Block 1049,G5040,R,None,None,S,130691,0,+38.5342417,-075.4535089,"POLYGON ((-75.45807 38.53492, -75.45771 38.535...",35,12,29
24113,10,005,051702,1120,100050517021120,Block 1120,G5040,R,None,None,S,155272,0,+38.5304973,-075.4586287,"POLYGON ((-75.45939 38.52753, -75.45941 38.527...",0,0,0


In [19]:
# Assign a fraction of the population of each block as a value to each point

# Merge each point to the 2010 census block containing it
population_per_point = point_cloud.merge(block_data, how="left", left_on="GEOID", right_on="GEOID10")

# Multiply each weight by the block population to get the block population per point
population_per_point[[x for x in population_per_point.columns if 'weight' in x]] = population_per_point[[x for x in population_per_point.columns if 'weight' in x]].mul(population_per_point["P001001"].astype(int), axis=0)

In [20]:
# Flatten to a GeoSeries where each row is a point and its weight
weights = np.array([[row["weight_" + str(i)] for i in range(n)] for _, row in population_per_point.iterrows()]).flatten()
points = np.array([[row["point_" + str(i)] for i in range(n)] for _, row in population_per_point.iterrows()]).flatten()
points_list = geopandas.GeoDataFrame({"population_per_point":weights,"geometry":points}, crs="EPSG:4269")


# Determine the number of points in the point cloud. This should be n * the number of census blocks
print(points_list.shape[0] / n == block_data.shape[0])

/Users/emiller/opt/miniconda3/envs/zip-tract-maps/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3378: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  exec(code_obj, self.user_global_ns, self.user_ns)
/var/folders/9r/738fljg527g64ktptsnnzlpw0000gn/T/ipykernel_1435/3707923690.py:3: FutureWarning: The input object of type 'Point' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Point', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  points = np.array([[row["point_" + str(i)] for i in range(n)] for _, row in population_per_point.iterrows()]).flatten()
/var/folders/9r/738fljg527

True


In [21]:
# Spatially join each point to the 2010 census tract containing it
variables_per_point = geopandas.sjoin(points_list, tract_data, how="left", op='within')

In [22]:
# WARNING: Plot is large and should only be rendered if necessary
# TODO: Points around the edge of the state are being lost

"""
# Find and plot all missed points 
missed_points = variables_per_point.loc[variables_per_point["index_right"].isna()]

# initialize the map and store it in a folium map object
us_map = folium.Map(location=[38.9108, -75.5277], zoom_start=8, tiles=None)

# Add background tiles
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(us_map)

# Style and highlight functions map population values to color values
style_function = lambda x: {"weight":0.5, 
                            'color':'black',
                            'fillColor':'red', 
                            'fillOpacity':0.75}

# Add a map over the tiles with the given colors and a tooltip
NIL=folium.features.GeoJson(
        missed_points, # Full geopandas data
        style_function=style_function, # function for base colors
        control=False
    )

# Add elements to map
us_map.add_child(NIL)"""

'\n# Find and plot all missed points \nmissed_points = variables_per_point.loc[variables_per_point["index_right"].isna()]\n\n# initialize the map and store it in a folium map object\nus_map = folium.Map(location=[38.9108, -75.5277], zoom_start=8, tiles=None)\n\n# Add background tiles\nfolium.TileLayer(\'CartoDB positron\',name="Light Map",control=False).add_to(us_map)\n\n# Style and highlight functions map population values to color values\nstyle_function = lambda x: {"weight":0.5, \n                            \'color\':\'black\',\n                            \'fillColor\':\'red\', \n                            \'fillOpacity\':0.75}\n\n# Add a map over the tiles with the given colors and a tooltip\nNIL=folium.features.GeoJson(\n        missed_points, # Full geopandas data\n        style_function=style_function, # function for base colors\n        control=False\n    )\n\n# Add elements to map\nus_map.add_child(NIL)'

In [23]:
# Exclude missed points from the list
variables_per_point = variables_per_point.loc[~variables_per_point["index_right"].isna()]

# Divide variables of intersest by tract population and multiply by the portion of the population represented by each point
variables_per_point[["P001001", "H001001", "P001003"]] = variables_per_point[["P001001", "H001001", "P001003"]].astype(int).div(variables_per_point["P001001"].astype(int), axis=0).mul(variables_per_point["population_per_point"], axis=0)
# Reset index
variables_per_point = variables_per_point.reset_index()

In [24]:
variables_per_point

,index,population_per_point,geometry,index_right,GEO_ID,STATE,COUNTY,TRACT,NAME,LSAD,CENSUSAREA,P001001,H001001,P001003
0,0,47.427880,POINT (-75.48698 39.12118),87.0,1400000US10001041100,10,001,041100,411,Tract,6.190,47.427880,14.545395,34.404235
1,1,49.453987,POINT (-75.48586 39.12186),87.0,1400000US10001041100,10,001,041100,411,Tract,6.190,49.453987,15.166771,35.873975
2,2,48.270648,POINT (-75.48673 39.12176),87.0,1400000US10001041100,10,001,041100,411,Tract,6.190,48.270648,14.803859,35.015580
3,3,51.011072,POINT (-75.48470 39.12196),87.0,1400000US10001041100,10,001,041100,411,Tract,6.190,51.011072,15.644305,37.003486
4,4,47.836413,POINT (-75.48680 39.12139),87.0,1400000US10001041100,10,001,041100,411,Tract,6.190,47.836413,14.670686,34.700585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119804,120570,0.000000,POINT (-75.46883 38.53035),176.0,1400000US10005051702,10,005,051702,517.02,Tract,68.279,0.000000,0.000000,0.000000
119805,120571,0.000000,POINT (-75.46198 38.52364),176.0,1400000US10005051702,10,005,051702,517.02,Tract,68.279,0.000000,0.000000,0.000000
119806,120572,0.000000,POINT (-75.45649 38.52289),176.0,1400000US10005051702,10,005,051702,517.02,Tract,68.279,0.000000,0.000000,0.000000
119807,120573,0.000000,POINT (-75.46826 38.52367),176.0,1400000US10005051702,10,005,051702,517.02,Tract,68.279,0.000000,0.000000,0.000000


In [25]:
# Print the number of points missed in the transfer of data from tracts to points
print(points_list.shape[0] -  variables_per_point.shape[0])

766


In [26]:
# Request shapefile data for 2020 census tracts and convert to geopandas dataframe

# Shapefile url
data_url = 'https://www2.census.gov/geo/tiger/GENZ2020/shp/cb_2020_10_tract_500k.zip'


# Request data
data = requests.get(data_url)
# convert to pandas dataframe
tract2020 = geopandas.read_file(BytesIO(data.content))

In [27]:
# Spatially join points to 2020 census tracts
interpolated_values = geopandas.sjoin(variables_per_point[["GEO_ID","geometry","P001001","H001001","P001003"]], tract2020, how="left", op='within')

# Sum the values for each 2020 tract
interpolated_values = interpolated_values[["GEOID", "P001001", "H001001", "P001003"]].groupby("GEOID").sum()

interpolated_values


#tract2020

,P001001,H001001,P001003
GEOID,,,
10001040100,6533.447483,2466.149188,5695.416228
10001040201,5033.370942,2019.835793,3659.283495
10001040203,5006.285861,2017.644878,3116.398468
10001040204,4666.379121,1737.049848,3168.077653
10001040205,2875.000000,1070.335851,1951.656666
...,...,...,...
10005051702,5597.842566,2296.772549,4999.458522
10005051801,4884.876548,2117.200173,3854.789459
10005051802,4189.572557,1756.284674,2519.962792


In [28]:
# Pull population data for 2020 Census tracts
# Define request parameters

year = '2020' # Year of interest
datasource = 'dec' # Survey name
subsource = 'pl' # Subsurvey name
GET = 'P1_001N,H1_001N,P1_003N' # Variables to query
FOR = 'tract:*' # for predicate
IN = 'state:10' # in predicate


# Filepath to your Census API key
keyfile = 'CensusAPIKey.txt'

# Formatted API call
data_url = f'https://api.census.gov/data/{year}/{datasource}/{subsource}?get={GET}&for={FOR}&in={IN}'

# Read Census key into 'api_key'
with open(keyfile) as key:
    api_key = key.read().strip()

# Add key to url
data_url = f'{data_url}&key={api_key}'

# Request data and convert from json
data = requests.get(data_url).json()
# First entry in list is a list of variable names
tract2020_data = pd.DataFrame(data[1:], columns = data[0])

# Add a GEOID column to the data
tract2020_data["GEOID"] = tract2020_data["state"].astype(str) + tract2020_data["county"].astype(str) +tract2020_data["tract"].astype(str)

In [29]:
# Write combined dataframe of 2020 ground truth and estimated values to a csv
interpolated_values.merge(tract2020_data, left_index=True, right_on="GEOID").to_csv("estimates.csv", index=False)

In [30]:
interpolated_values.merge(tract2020_data, left_index=True, right_on="GEOID")

,P001001,H001001,P001003,P1_001N,H1_001N,P1_003N,state,county,tract,GEOID
220,6533.447483,2466.149188,5695.416228,7315,2740,5980,10,001,040100,10001040100
221,5033.370942,2019.835793,3659.283495,5446,2123,3424,10,001,040201,10001040201
222,5006.285861,2017.644878,3116.398468,5182,2157,2808,10,001,040203,10001040203
223,4666.379121,1737.049848,3168.077653,6451,2269,3613,10,001,040204,10001040204
224,2875.000000,1070.335851,1951.656666,4699,1985,2430,10,001,040205,10001040205
...,...,...,...,...,...,...,...,...,...,...
214,5597.842566,2296.772549,4999.458522,6577,2590,5286,10,005,051702,10005051702
215,4884.876548,2117.200173,3854.789459,5359,2154,3636,10,005,051801,10005051801
216,4189.572557,1756.284674,2519.962792,4354,1740,2256,10,005,051802,10005051802
217,4562.095370,1831.833694,3629.689021,4760,1949,3566,10,005,051900,10005051900
